# Modelo hibrido de recomendación para B2C

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.sparse import lil_matrix, csr_matrix
from collections import defaultdict
import time

# Carga de Datos

In [2]:
direccion_B2C_transacciones = 'Datos/transacciones_con_features.csv'
direccion_B2C_cotizaciones = 'Datos/cotizaciones_con_features.csv'
transacciones = pd.read_csv(direccion_B2C_transacciones, encoding='utf-8')
cotizaciones = pd.read_csv(direccion_B2C_cotizaciones, encoding='utf-8')


In [3]:
print(f"Tamaño transacciones: {transacciones.shape}")
print(f"Tamaño cotizaciones: {cotizaciones.shape}")

Tamaño transacciones: (2099287, 43)
Tamaño cotizaciones: (178378, 22)


In [4]:
cotizaciones.head(5)

,cotizacion,estado_cotizacion,id,fecha_creacion,fecha_modificacion,categoria_macro,categoria,producto,cantidad,precio,...,mes_cot,dia_semana_cot,hora_cot,dias_modificacion_cot,total_unidades_cotizadas,valor_total_cotizado,n_cotizaciones_producto,n_clientes_cotizaron,tasa_conversion_cot_prod,producto_fue_comprado_por_cliente
0,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_9,producto_524,1.0,34.957429,...,8,5,12,0,131.0,4602.043186,88,78,0.647727,1
1,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_11,producto_124,1.0,2.262000,...,8,5,12,0,3113.0,7071.418815,1768,1585,0.802036,1
2,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_10,producto_5085,1.0,40.600000,...,8,5,12,0,12.0,490.141515,12,9,0.750000,1
3,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_11,producto_188,1.0,7.863143,...,8,5,12,0,4635.0,36578.698818,2768,2541,0.803829,1
4,1381864.0,Cotización Ganada,93623,1972-08-05 14:36:57.273,1972-08-05 15:06:26,categoria_macro_4,categoria_10,producto_391,1.0,63.932571,...,8,5,14,0,349.0,22408.407683,294,269,0.785714,1


In [5]:
transacciones.head(5)

,fecha,pedido,id,edad,municipio,zona,asesor,punto de venta,cluster,categoria_macro,...,n_categorias_distintas_cliente,gasto_promedio_pedido_cliente,frecuencia_compra_cliente,n_transacciones_cat,n_clientes_cat,n_productos_unicos_cat,popularidad_valor_prod_en_cat,popularidad_unidad_prod_en_cat,popularidad_valor_prod_global,popularidad_unidad_prod_global
0,1971-04-30,2,2,52,EL CARMEN DE CHUCURI,SANTANDER,asesor_2,punto_venta_2,cluster_tienda_2,categoria_macro_1,...,5,32.427500,0.027972,34036,25877,86,0.057429,0.003877,0.000435,0.000014
1,1971-04-30,3,3,31,VILLANUEVA,LA GUAJIRA,asesor_3,punto_venta_2,cluster_tienda_2,categoria_macro_2,...,1,1.130000,1.000000,280686,173251,93,0.004349,0.001100,0.000403,0.000730
2,1971-04-30,4,4,43,VILLANUEVA,LA GUAJIRA,asesor_4,punto_venta_2,cluster_tienda_2,categoria_macro_3,...,1,8.380000,1.000000,27271,23506,175,0.008657,0.008289,0.000091,0.000012
3,1971-04-30,5,5,31,VILLANUEVA,LA GUAJIRA,asesor_5,punto_venta_3,cluster_tienda_3,categoria_macro_2,...,4,24.500000,0.038961,238206,146629,426,0.049078,0.072158,0.007394,0.003558
4,1971-04-30,6,6,49,ARROYOHONDO,BOLÍVAR,asesor_6,punto_venta_4,cluster_tienda_2,categoria_macro_4,...,7,49.499474,0.029874,30515,25309,298,0.016124,0.014238,0.000191,0.000019


In [6]:
numeric_columns = transacciones.select_dtypes(include=['number']).columns
print(numeric_columns)

Index(['pedido', 'id', 'edad', 'cantidad', 'precio', 'valor',
       'alineación con portafolio estratégico', 'año_venta', 'mes_venta',
       'dia_semana_venta', 'dia_mes_venta', 'semana_año_venta',
       'total_unidades_vendidas', 'valor_total_ventas',
       'precio_promedio_venta', 'n_transacciones_producto',
       'n_clientes_producto', 'frecuencia_venta_prod', 'recency',
       'n_pedidos_cliente', 'monetary', 'n_items_distintos_cliente',
       'n_categorias_distintas_cliente', 'gasto_promedio_pedido_cliente',
       'frecuencia_compra_cliente', 'n_transacciones_cat', 'n_clientes_cat',
       'n_productos_unicos_cat', 'popularidad_valor_prod_en_cat',
       'popularidad_unidad_prod_en_cat', 'popularidad_valor_prod_global',
       'popularidad_unidad_prod_global'],
      dtype='object')


# 1. Selección y Agregación de Features por Producto 
Se seleccionan columnas clave de transacciones y cotizaciones que describen al producto y su comportamiento (ventas, cotizaciones, categorías, etc.), asegurando que 'producto' esté presente para agrupar la información. Luego, se agrupan los datos por producto y se agregan usando funciones como media o primera ocurrencia, generando un resumen por producto que puede usarse en modelos o análisis posteriores.


In [ ]:
# Añadimos 'producto' para la agrupación y el índice


relevant_cols_transacciones_user = ['producto'] + [
    'categoria_macro', 'categoria', 'subcategoria', 'color',
    'precio_promedio_venta', 'alineación con portafolio estratégico',
    'total_unidades_vendidas', 'valor_total_ventas', 'n_transacciones_producto',
    'n_clientes_producto', 'frecuencia_venta_prod', 'popularidad_valor_prod_en_cat',
    'popularidad_unidad_prod_en_cat', 'popularidad_valor_prod_global',
    'popularidad_unidad_prod_global'
]

relevant_cols_cotizaciones_user = ['producto'] + [
    'categoria_macro', 'categoria',
    'total_unidades_cotizadas', 'valor_total_cotizado',
    'n_cotizaciones_producto', 'producto_fue_comprado_por_cliente' # Agregaremos esto
]

# Verificar que 'producto' existe
if 'producto' not in transacciones.columns or 'producto' not in cotizaciones.columns:
    print("ERROR: La columna 'producto' no se encuentra en uno o ambos DataFrames.")
    exit()

# Filtrar columnas que realmente existen en los dataframes cargados
available_cols_trans = [col for col in relevant_cols_transacciones_user if col in transacciones.columns]
available_cols_cot = [col for col in relevant_cols_cotizaciones_user if col in cotizaciones.columns]

print(f"Columnas disponibles y seleccionadas de Transacciones: {len(available_cols_trans)}")
print(f"Columnas disponibles y seleccionadas de Cotizaciones: {len(available_cols_cot)}")

# Agregar datos para tener una fila por producto

# Transacciones: Agregamos columnas que podrían variar por transacción (color, alineación)
# y tomamos el valor medio/moda. Las features pre-agregadas ('total_unidades_vendidas', etc.)
# deberían ser consistentes, usamos 'mean' como método seguro.
trans_agg_dict = {
    # Categóricas: tomar la más frecuente (moda) o la primera si solo hay una
    'categoria_macro': 'first',
    'categoria': 'first',
    'subcategoria': 'first',
    'color': lambda x: x.mode()[0] if not x.mode().empty else 'Desconocido',
    # Numéricas: usar la media. Para las pre-agregadas, esto no debería cambiar el valor.
    'precio_promedio_venta': 'mean',
    'alineación con portafolio estratégico': 'mean',
    'total_unidades_vendidas': 'mean',
    'valor_total_ventas': 'mean',
    'n_transacciones_producto': 'mean',
    'n_clientes_producto': 'mean',
    'frecuencia_venta_prod': 'mean',
    'popularidad_valor_prod_en_cat': 'mean',
    'popularidad_unidad_prod_en_cat': 'mean',
    'popularidad_valor_prod_global': 'mean',
    'popularidad_unidad_prod_global': 'mean'
}
# Filtrar el diccionario de agregación para usar solo columnas disponibles
valid_trans_agg_dict = {k: v for k, v in trans_agg_dict.items() if k in available_cols_trans}
product_features_trans = transacciones.groupby('producto').agg(valid_trans_agg_dict)
print(f"Features agregadas por producto de Transacciones: {product_features_trans.shape}")

# Cotizaciones: Similar, agregamos flag y calculamos precio/valor promedio de cotización
cot_agg_dict = {
    'categoria_macro': 'first',
    'categoria': 'first',
    'precio': 'mean', # Precio promedio al que se cotiza este producto
    'valor': 'mean',  # Valor promedio (precio*cantidad) de la línea de cotización
    'total_unidades_cotizadas': 'mean', # Tomar el valor pre-calculado
    'valor_total_cotizado': 'mean', # Tomar el valor pre-calculado
    'n_cotizaciones_producto': 'mean', # Tomar el valor pre-calculado
    'producto_fue_comprado_por_cliente': 'max' # 1 si alguna vez fue comprado post-cotización, 0 si no
}
# Filtrar columnas disponibles y crear diccionario de agregación válido
cols_for_cot_agg = [col for col in available_cols_cot if col != 'producto'] # Excluir 'producto' de las claves de agg
# Añadir precio y valor originales si no están pero se usan en el dict
if 'precio' not in cols_for_cot_agg and 'precio' in cotizaciones.columns: cols_for_cot_agg.append('precio')
if 'valor' not in cols_for_cot_agg and 'valor' in cotizaciones.columns: cols_for_cot_agg.append('valor')

valid_cot_agg_dict = {k: v for k, v in cot_agg_dict.items() if k in cols_for_cot_agg or k in available_cols_cot} # Asegurar que usemos las columnas seleccionadas por el usuario
# Quitar keys del dict si la columna original no existe en cotizaciones
valid_cot_agg_dict = {k: v for k, v in valid_cot_agg_dict.items() if k in cotizaciones.columns or k in available_cols_cot}

product_features_cot = cotizaciones.groupby('producto').agg(valid_cot_agg_dict)
# Renombrar precio/valor promedio de cotización para claridad
product_features_cot = product_features_cot.rename(columns={'precio': 'precio_promedio_cot', 'valor': 'valor_promedio_cot'})
print(f"Features agregadas por producto de Cotizaciones: {product_features_cot.shape}")



Columnas disponibles y seleccionadas de Transacciones: 16
Columnas disponibles y seleccionadas de Cotizaciones: 7
Features agregadas por producto de Transacciones: (7276, 15)
Features agregadas por producto de Cotizaciones: (2735, 8)


# 2. Unificar Features de Producto
Se crea un índice unificado para todos los productos encontrados, generando dos mapeos: de producto a índice y de índice a producto. Esto permite referenciar eficientemente los productos en estructuras numéricas como matrices. Finalmente, se reindexa el DataFrame unificado para asegurar que todos los productos estén correctamente ordenados y alineados.


In [ ]:
# Eliminar las columnas duplicadas del dataframe de cotizaciones
product_features_cot_1 = product_features_cot.drop(columns=['categoria_macro', 'categoria'])

product_features_unified = product_features_trans.join(product_features_cot_1, how='outer')
print(f"Shape unificado inicial: {product_features_unified.shape}")
product_features_unified.head()


Shape unificado inicial: (7277, 21)


,categoria_macro,categoria,subcategoria,color,precio_promedio_venta,alineación con portafolio estratégico,total_unidades_vendidas,valor_total_ventas,n_transacciones_producto,n_clientes_producto,...,popularidad_valor_prod_en_cat,popularidad_unidad_prod_en_cat,popularidad_valor_prod_global,popularidad_unidad_prod_global,precio_promedio_cot,valor_promedio_cot,total_unidades_cotizadas,valor_total_cotizado,n_cotizaciones_producto,producto_fue_comprado_por_cliente
producto,,,,,,,,,,,,,,,,,,,,,
producto_1,categoria_macro_1,categoria_1,subcategoria_1,MATE,8.567222,1.076122,134.00,1151.24,90.0,69.0,...,0.000528,0.000107,0.000014,0.000002,8.749715,10.208000,14.0,122.496006,12.0,1.0
producto_10,categoria_macro_1,categoria_1,subcategoria_7,BLANCO,11.459514,1.784772,1984.00,22600.24,1291.0,1058.0,...,0.010373,0.001582,0.000269,0.000025,11.851216,38.529133,1143.0,13639.313110,349.0,1.0
producto_100,categoria_macro_4,categoria_11,subcategoria_12,No encontrado,1.919338,0.837496,1446.00,2467.46,302.0,281.0,...,0.002205,0.003688,0.000029,0.000018,2.021534,11.929538,543.0,1097.517518,91.0,1.0
producto_1000,categoria_macro_2,categoria_7,subcategoria_39,No encontrado,10.784545,11.038150,98.82,1072.15,11.0,9.0,...,0.000033,0.000010,0.000013,0.000001,NaN,NaN,NaN,NaN,NaN,NaN
producto_1001,categoria_macro_2,categoria_7,subcategoria_39,No encontrado,4.975563,9.415715,7885.80,38550.79,458.0,370.0,...,0.001176,0.000835,0.000459,0.000099,NaN,NaN,NaN,NaN,NaN,NaN


# 3. Crear Índice Unificado y Mapeos
Para unificar las características por producto, primero se eliminan columnas duplicadas como `categoria_macro` y `categoria` del DataFrame de cotizaciones. Luego, se realiza un `join` entre los datos agregados de transacciones y cotizaciones usando el identificador de producto como índice. El resultado es un DataFrame unificado con información combinada de ambos orígenes.



In [ ]:

print("\n--- 3. Creando Índice Unificado ---")
all_unique_products = product_features_unified.index.unique().tolist()
product_to_idx = {product: i for i, product in enumerate(all_unique_products)}
idx_to_product = {i: product for product, i in product_to_idx.items()}
n_products = len(all_unique_products)
print(f"Total de productos únicos encontrados: {n_products}")

# Reindexar por si acaso (aunque el join ya debería tener el índice correcto)
product_features_unified = product_features_unified.reindex(all_unique_products)



--- 3. Creando Índice Unificado ---
Total de productos únicos encontrados: 7277


# 4. Imputar Valores Faltantes (NaN)

En esta etapa, se imputan los valores faltantes tanto en columnas numéricas como categóricas del DataFrame unificado. Para las columnas numéricas se utiliza la mediana y para las categóricas la moda, garantizando que no queden `NaNs` en los datos antes del preprocesamiento.


In [ ]:
# Identificar columnas numéricas y categóricas en el DF unificado
final_num_features = product_features_unified.select_dtypes(include=np.number).columns.tolist()
final_cat_features = product_features_unified.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Columnas numéricas para imputar/escalar: {len(final_num_features)}")
# print(final_num_features)
print(f"Columnas categóricas para imputar/codificar: {len(final_cat_features)}")
# print(final_cat_features)

imputation_count = 0
for col in final_num_features:
    if product_features_unified[col].isnull().any():
        imputation_count += 1
        median_val = product_features_unified[col].median()
        fill_val = median_val if pd.notna(median_val) else 0 
        product_features_unified[col] = product_features_unified[col].fillna(fill_val)

for col in final_cat_features:
    if product_features_unified[col].isnull().any():
        imputation_count += 1
        mode_val = product_features_unified[col].mode()
        fill_value = mode_val[0] if not mode_val.empty else 'Desconocido'
        product_features_unified[col] = product_features_unified[col].fillna(fill_value)

if imputation_count > 0:
    print(f"Se imputaron NaNs en {imputation_count} instancias de columna.")
else:
    print("No se encontraron NaNs para imputar.")

# Verificar que no queden NaNs
nans_remaining = product_features_unified.isnull().sum().sum()
if nans_remaining > 0:
    print(f"ADVERTENCIA: ¡Todavía quedan {nans_remaining} NaNs después de la imputación!")
    # print(product_features_unified.isnull().sum()[product_features_unified.isnull().sum() > 0])
else:
    print("Verificación de NaNs completada: No hay NaNs restantes.")


--- 4. Imputando Valores Faltantes ---
Columnas numéricas para imputar/escalar: 17
Columnas categóricas para imputar/codificar: 4
Se imputaron NaNs en 21 instancias de columna.
Verificación de NaNs completada: No hay NaNs restantes.


In [11]:
product_features_unified.sample(10)

,categoria_macro,categoria,subcategoria,color,precio_promedio_venta,alineación con portafolio estratégico,total_unidades_vendidas,valor_total_ventas,n_transacciones_producto,n_clientes_producto,...,popularidad_valor_prod_en_cat,popularidad_unidad_prod_en_cat,popularidad_valor_prod_global,popularidad_unidad_prod_global,precio_promedio_cot,valor_promedio_cot,total_unidades_cotizadas,valor_total_cotizado,n_cotizaciones_producto,producto_fue_comprado_por_cliente
producto,,,,,,,,,,,,,,,,,,,,,
producto_5124,categoria_macro_4,categoria_6,subcategoria_6,No encontrado,1.587000,0.260928,14.00,21.68,10.0,10.0,...,0.000022,0.000131,2.581717e-07,1.752112e-07,9.354571,26.783295,21.0,240.965146,7.0,1.0
producto_980,categoria_macro_4,categoria_9,subcategoria_10,No encontrado,12.307574,3.027206,1461.00,17289.82,705.0,672.0,...,0.006085,0.007722,2.058922e-04,1.828454e-05,12.956420,58.759421,615.0,7991.281301,135.0,1.0
producto_4065,categoria_macro_4,categoria_21,subcategoria_58,No encontrado,205.440000,30.988656,12.00,2518.32,8.0,6.0,...,0.001084,0.000238,2.998888e-05,1.501811e-07,9.354571,26.783295,21.0,240.965146,7.0,1.0
producto_2557,categoria_macro_3,categoria_16,subcategoria_33,BLANCO,2.587500,2.308032,178.00,431.23,12.0,10.0,...,0.001292,0.001376,5.135212e-06,2.227686e-06,3.206572,25.011258,39.0,125.056292,5.0,1.0
producto_2528,categoria_macro_3,categoria_17,subcategoria_69,MADERA,0.175714,83.433929,61071.00,10631.28,21.0,21.0,...,0.006403,0.008857,1.266004e-04,7.643089e-04,0.197200,1087.300783,126815.0,25007.918000,22.0,1.0
producto_3650,categoria_macro_2,categoria_7,subcategoria_64,No encontrado,5.485000,1.767168,22.92,127.34,6.0,4.0,...,0.000004,0.000002,1.516402e-06,2.868458e-07,9.354571,26.783295,21.0,240.965146,7.0,1.0
producto_1289,categoria_macro_2,categoria_7,subcategoria_5,GRIS,2.263235,1.728881,977.60,2179.55,102.0,95.0,...,0.000066,0.000103,2.595471e-05,1.223475e-05,9.354571,26.783295,21.0,240.965146,7.0,1.0
producto_1245,categoria_macro_1,categoria_1,subcategoria_42,No encontrado,1.620000,0.094125,17.00,27.54,17.0,17.0,...,0.000013,0.000014,3.279543e-07,2.127565e-07,1.615714,1.615714,4.0,6.462855,4.0,1.0
producto_512,categoria_macro_4,categoria_9,subcategoria_25,No encontrado,26.388221,4.470959,595.00,15671.37,444.0,423.0,...,0.005516,0.003145,1.866192e-04,7.446477e-06,27.747667,39.029961,225.0,6244.793819,158.0,1.0


# 5. Preprocesamiento (Escalado y Codificación)

Se normalizan las columnas numéricas con `MinMaxScaler` y se codifican las categóricas usando `OneHotEncoder`, generando una matriz de características dispersa que resume toda la información de productos de forma estructurada y lista para el cálculo de similitud.


In [ ]:
start_time_preprocess = time.time()

# Crear el ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), final_num_features),
        # Usar sparse_output=True para eficiencia con muchas categorías
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=True), final_cat_features)
    ],
    remainder='drop' # Ignorar columnas no especificadas (como el índice si no se quitó)
)

# Ajustar y transformar los datos unificados e imputados
try:
    feature_matrix_sparse = preprocessor.fit_transform(product_features_unified)
    end_time_preprocess = time.time()
    print(f"Matriz de características dispersa creada: {feature_matrix_sparse.shape}")
    print(f"Preprocesamiento completado en {end_time_preprocess - start_time_preprocess:.2f} segundos.")
    preprocess_ok = True
except Exception as e:
    print(f"ERROR durante el preprocesamiento: {e}")
    preprocess_ok = False
    feature_matrix_sparse = None


Matriz de características dispersa creada: (7277, 220)
Preprocesamiento completado en 0.03 segundos.


# 6. Cálculo de Similitud de Contenido

Usando la matriz de características procesada, se calcula una matriz de similitud basada en la métrica de coseno. Esta matriz permite cuantificar qué tan similares son los productos entre sí con base en sus características.


In [ ]:

content_similarity_matrix = None
similarity_ok = False
if preprocess_ok and feature_matrix_sparse is not None:
    print("\n--- 6. Calculando Similitud Coseno ---")
    start_time_similarity = time.time()
    try:
        # Asegurarse que la matriz no esté vacía
        if feature_matrix_sparse.shape[0] > 0 and feature_matrix_sparse.shape[1] > 0:
             content_similarity_matrix = cosine_similarity(feature_matrix_sparse)
             end_time_similarity = time.time()
             print(f"Matriz de similitud coseno calculada: {content_similarity_matrix.shape}")
             print(f"Cálculo de similitud completado en {end_time_similarity - start_time_similarity:.2f} segundos.")
             similarity_ok = True
        else:
             print("ERROR: La matriz de características está vacía o tiene dimensiones cero.")

    except Exception as e:
        print(f"ERROR durante el cálculo de similitud: {e}")
        content_similarity_matrix = None # Asegurar que sea None si falla
else:
    print("\nSe omite el cálculo de similitud debido a un error en el preprocesamiento.")



--- 6. Calculando Similitud Coseno ---
Matriz de similitud coseno calculada: (7277, 7277)
Cálculo de similitud completado en 1.82 segundos.


# 7. Función de Recomendación Basada en Contenido

Se implementa una función que, dado un producto de entrada, devuelve una lista de productos similares ordenados por su score de similitud coseno. Esta función permite generar recomendaciones personalizadas sin necesidad de datos de usuario.


In [ ]:

def get_content_recommendations(input_product, N=10):
    """
    Genera recomendaciones basadas puramente en la similitud de contenido.

    Args:
        input_product (str): El ID del producto para el que se quieren recomendaciones.
        N (int): Número de recomendaciones a generar.

    Returns:
        pandas.DataFrame: DataFrame con 'producto' recomendado y 'similarity_score'.
                          Devuelve DataFrame vacío si el producto no se encuentra o hay error.
    """
    recommendations = pd.DataFrame()
    # Verificar si la matriz de similitud está disponible
    if not similarity_ok or content_similarity_matrix is None:
        print("Error: Matriz de similitud no disponible.")
        return recommendations

    # Verificar si el producto de entrada existe en nuestro mapeo
    if input_product not in product_to_idx:
        print(f"Error: Producto '{input_product}' no encontrado en el mapeo.")
        return recommendations

    # Obtener el índice numérico del producto de entrada
    idx = product_to_idx[input_product]

    # Obtener los scores de similitud para este producto (fila 'idx' de la matriz)
    # enumerate añade un contador a los scores para saber su índice original
    sim_scores = list(enumerate(content_similarity_matrix[idx]))

    # Ordenar los productos basados en el score de similitud (descendente)
    # El score está en la posición 1 de cada tupla (indice, score)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener los N productos más similares (excluyendo el propio producto)
    top_recommendations = []
    for i, score in sim_scores:
        # Omitir el producto de entrada (su índice es 'idx')
        if i == idx:
            continue

        # Obtener el nombre del producto usando el mapeo inverso
        recommended_product_id = idx_to_product.get(i)

        # Añadir a la lista si encontramos el nombre y no hemos alcanzado N
        if recommended_product_id and len(top_recommendations) < N:
             top_recommendations.append({'producto': recommended_product_id, 'similarity_score': score})
        elif len(top_recommendations) >= N:
            break # Salir del bucle si ya tenemos N recomendaciones

    if not top_recommendations:
        print(f"No se encontraron recomendaciones para '{input_product}' (excluyéndose a sí mismo).")
    else:
        recommendations = pd.DataFrame(top_recommendations)

    return recommendations


# 8. Ejemplo de Uso

Se muestra un ejemplo práctico donde se obtienen recomendaciones de productos similares para un producto específico. Si la matriz de similitud fue calculada correctamente, se imprimen los resultados y el tiempo que tomó generar las recomendaciones.

In [ ]:
if similarity_ok and all_unique_products:
    
    example_product_id = "producto_125"

    if example_product_id:
        print(f"Obteniendo recomendaciones de contenido para: '{example_product_id}'")
        start_rec_time = time.time()
        recs = get_content_recommendations(example_product_id, N=5)
        end_rec_time = time.time()

        if not recs.empty:
            print(recs)
            print(f"\nRecomendaciones generadas en {end_rec_time - start_rec_time:.4f} segundos.")
        else:
            print("No se generaron recomendaciones para el producto de ejemplo.")


else:
    print("No se puede ejecutar el ejemplo: la matriz de similitud no está lista o no hay productos.")

print("\n--- Proceso de Modelo de Contenido Completado ---")

Obteniendo recomendaciones de contenido para: 'producto_125'
        producto  similarity_score
0   producto_634          0.999970
1   producto_364          0.999912
2  producto_3106          0.999847
3   producto_393          0.999708
4   producto_419          0.999657

Recomendaciones generadas en 0.0040 segundos.

--- Proceso de Modelo de Contenido Completado ---


# 9. Cálculo de Co-ocurrencia (Co-Compra - Transacciones)

En este paso, se calcula una matriz de co-ocurrencia para las transacciones de productos. Este proceso se realiza agrupando los productos por pedido, incrementando el contador en la matriz cada vez que dos productos aparecen en el mismo pedido. El resultado es una matriz dispersa que representa la co-compra entre productos.


In [ ]:

start_time_cf_buy = time.time()
co_occurrence_matrix = None # Inicializar
cf_buy_ok = False

# Verificar si 'pedido' y 'producto' existen y si hay mapeos
if 'pedido' in transacciones.columns and 'producto' in transacciones.columns and product_to_idx:
    # Crear matriz de co-ocurrencia dispersa (LIL para construcción)
    co_occurrence_matrix_lil = lil_matrix((n_products, n_products), dtype=np.int32)

    # Agrupar por pedido y obtener la lista de productos únicos en cada pedido
    # Asegurarse de que solo procesamos productos conocidos en nuestro mapeo
    pedidos_grouped = transacciones[transacciones['producto'].isin(product_to_idx.keys())].groupby('pedido')['producto'].unique()

    processed_pairs_buy = 0
    # Iterar sobre los pedidos para llenar la matriz
    for products_in_pedido in pedidos_grouped:
        # Obtener los índices numéricos de los productos en este pedido
        indices_in_pedido = [product_to_idx[p] for p in products_in_pedido] # Ya filtramos productos desconocidos

        # Incrementar el contador para cada par de productos distintos en el pedido
        for i in range(len(indices_in_pedido)):
            for j in range(i + 1, len(indices_in_pedido)):
                idx1, idx2 = indices_in_pedido[i], indices_in_pedido[j]
                # Incrementar en ambas direcciones (matriz simétrica)
                co_occurrence_matrix_lil[idx1, idx2] += 1
                co_occurrence_matrix_lil[idx2, idx1] += 1
                processed_pairs_buy += 1 # Contar cada par único

    # Convertir a CSR para operaciones matemáticas más rápidas después
    co_occurrence_matrix_csr = co_occurrence_matrix_lil.tocsr()

    end_time_cf_buy = time.time()
    print(f"Matriz de co-compra (dispersa {co_occurrence_matrix_csr.shape}) calculada en {end_time_cf_buy - start_time_cf_buy:.2f} seg.")
    if n_products > 0:
        print(f"Total de pares únicos co-comprados encontrados: {processed_pairs_buy}")
        print(f"Densidad Co-compra: {co_occurrence_matrix_csr.nnz / (n_products * n_products):.6f}")
    else:
        print("No hay productos para calcular densidad.")
    cf_buy_ok = True
    co_occurrence_matrix = co_occurrence_matrix_csr # Asignar la matriz CSR final

else:
    print("Advertencia: No se puede calcular co-compra. Falta 'pedido'/'producto' en transacciones o mapeo de productos.")
    co_occurrence_matrix = None



Matriz de co-compra (dispersa (7277, 7277)) calculada en 78.22 seg.
Total de pares únicos co-comprados encontrados: 3764964
Densidad Co-compra: 0.025644


# 10. Cálculo de Co-ocurrencia (Co-Cotización - Cotizaciones)

Al igual que con las transacciones, se calcula una matriz de co-ocurrencia para las cotizaciones de productos. Se agrupan los productos por cotización y se actualiza la matriz de co-ocurrencia cada vez que dos productos son cotizados juntos. Este enfoque genera una matriz dispersa que representa las co-cotizaciones entre productos.


In [ ]:

start_time_cf_quote = time.time()
co_quotation_matrix = None # Inicializar
cf_quote_ok = False

# Verificar si 'cotizacion' y 'producto' existen y si hay mapeos
if 'cotizacion' in cotizaciones.columns and 'producto' in cotizaciones.columns and product_to_idx:
    # Crear matriz dispersa
    co_quotation_matrix_lil = lil_matrix((n_products, n_products), dtype=np.int32)

    # Agrupar por cotización y obtener productos únicos (filtrando desconocidos)
    cotizaciones_grouped = cotizaciones[cotizaciones['producto'].isin(product_to_idx.keys())].groupby('cotizacion')['producto'].unique()

    processed_pairs_quote = 0
    # Iterar sobre las cotizaciones
    for products_in_quote in cotizaciones_grouped:
        # Obtener índices numéricos (usando el mapeo unificado)
        indices_in_quote = [product_to_idx[p] for p in products_in_quote]
        # Incrementar contador para cada par distinto
        for i in range(len(indices_in_quote)):
            for j in range(i + 1, len(indices_in_quote)):
                idx1, idx2 = indices_in_quote[i], indices_in_quote[j]
                co_quotation_matrix_lil[idx1, idx2] += 1
                co_quotation_matrix_lil[idx2, idx1] += 1
                processed_pairs_quote += 1

    # Convertir a CSR
    co_quotation_matrix_csr = co_quotation_matrix_lil.tocsr()

    end_time_cf_quote = time.time()
    print(f"Matriz de co-cotización (dispersa {co_quotation_matrix_csr.shape}) calculada en {end_time_cf_quote - start_time_cf_quote:.2f} seg.")
    if n_products > 0:
        print(f"Total de pares únicos co-cotizados encontrados: {processed_pairs_quote}")
        print(f"Densidad Co-cotización: {co_quotation_matrix_csr.nnz / (n_products * n_products):.6f}")
    else:
        print("No hay productos para calcular densidad.")
    cf_quote_ok = True
    co_quotation_matrix = co_quotation_matrix_csr # Asignar la matriz CSR final
else:
    print("Advertencia: No se puede calcular co-cotización. Falta 'cotizacion'/'producto' en cotizaciones o mapeo de productos.")
    co_quotation_matrix = None


Matriz de co-cotización (dispersa (7277, 7277)) calculada en 9.62 seg.
Total de pares únicos co-cotizados encontrados: 249001
Densidad Co-cotización: 0.002628


# 11. Funciones de Recomendación Basadas en Co-ocurrencia

Se implementan funciones que utilizan las matrices de co-ocurrencia para generar recomendaciones basadas en co-compra y co-cotización. Estas funciones emplean las matrices dispersas generadas en los pasos anteriores para identificar productos que han sido comprados o cotizados juntos y los ordenan en función de la frecuencia de co-ocurrencia.


In [ ]:

def get_co_purchase_recommendations(input_product, N=10):
    """
    Genera recomendaciones basadas *solo* en co-compras.

    Args:
        input_product (str): El ID del producto de entrada.
        N (int): Número de recomendaciones.

    Returns:
        pandas.DataFrame: Top N productos co-comprados con 'producto' y 'co_purchase_count'.
                          DF vacío si hay error o no hay co-compras.
    """
    recommendations = pd.DataFrame()
    if not cf_buy_ok or co_occurrence_matrix is None:
        print("Error: Matriz de co-compra no disponible.")
        return recommendations
    if input_product not in product_to_idx:
        print(f"Error: Producto '{input_product}' no encontrado en el mapeo.")
        return recommendations

    idx = product_to_idx[input_product]

    try:
        # Obtener la fila de co-ocurrencias (matriz CSR)
        co_buys = co_occurrence_matrix[idx, :]
        # Obtener índices y valores de las columnas no cero
        co_bought_indices = co_buys.indices
        co_bought_values = co_buys.data

        if len(co_bought_indices) > 0:
            # Crear lista de tuplas (producto, count)
            recs_list = []
            for i, count in zip(co_bought_indices, co_bought_values):
                if i != idx: # Excluir el producto mismo
                    prod_name = idx_to_product.get(i)
                    if prod_name:
                        recs_list.append({'producto': prod_name, 'co_purchase_count': count})

            if recs_list:
                recommendations = pd.DataFrame(recs_list)
                recommendations = recommendations.sort_values('co_purchase_count', ascending=False).head(N)
            else:
                 print(f"No se encontraron co-compras (excluyendo self) para '{input_product}'.")
        else:
            print(f"Producto '{input_product}' no tiene co-compras registradas.")

    except Exception as e:
        print(f"Error obteniendo recomendaciones de co-compra: {e}")

    return recommendations


def get_co_quotation_recommendations(input_product, N=10):
    """
    Genera recomendaciones basadas *solo* en co-cotizaciones.

    Args:
        input_product (str): El ID del producto de entrada.
        N (int): Número de recomendaciones.

    Returns:
        pandas.DataFrame: Top N productos co-cotizados con 'producto' y 'co_quotation_count'.
                          DF vacío si hay error o no hay co-cotizaciones.
    """
    recommendations = pd.DataFrame()
    if not cf_quote_ok or co_quotation_matrix is None:
        print("Error: Matriz de co-cotización no disponible.")
        return recommendations
    if input_product not in product_to_idx:
        print(f"Error: Producto '{input_product}' no encontrado en el mapeo.")
        return recommendations

    idx = product_to_idx[input_product]

    try:
        # Obtener la fila de co-cotizaciones (matriz CSR)
        co_quotes = co_quotation_matrix[idx, :]
        co_quote_indices = co_quotes.indices
        co_quote_values = co_quotes.data

        if len(co_quote_indices) > 0:
             recs_list = []
             for i, count in zip(co_quote_indices, co_quote_values):
                 if i != idx: # Excluir self
                     prod_name = idx_to_product.get(i)
                     if prod_name:
                         recs_list.append({'producto': prod_name, 'co_quotation_count': count})

             if recs_list:
                recommendations = pd.DataFrame(recs_list)
                recommendations = recommendations.sort_values('co_quotation_count', ascending=False).head(N)
             else:
                 print(f"No se encontraron co-cotizaciones (excluyendo self) para '{input_product}'.")

        else:
            print(f"Producto '{input_product}' no tiene co-cotizaciones registradas.")

    except Exception as e:
        print(f"Error obteniendo recomendaciones de co-cotización: {e}")

    return recommendations


# 12. Ejemplo de Uso de Recomendaciones

Aquí se muestra un ejemplo de cómo utilizar las funciones de recomendación para obtener productos similares basados en co-compra. En este ejemplo, se toma un producto de entrada y se obtienen las recomendaciones basadas en la matriz de co-ocurrencia calculada previamente. Además, se imprime el tiempo que tomó generar las recomendaciones.


In [ ]:

# Usar el mismo producto de ejemplo que en el paso anterior si es posible
example_product_id = 'producto_125'
print(f"\nObteniendo recomendaciones CF para: '{example_product_id}'")

# Ejemplo Co-Compra
if cf_buy_ok:
    print("\n--- Recomendaciones por Co-Compra ---")
    start_rec_time_buy = time.time()
    recs_buy = get_co_purchase_recommendations(example_product_id, N=5)
    end_rec_time_buy = time.time()
    if not recs_buy.empty:
        print(recs_buy)
        print(f"Tiempo: {end_rec_time_buy - start_rec_time_buy:.4f} seg.")
    else:
        print("No se encontraron recomendaciones.")

# Ejemplo Co-Cotización
if cf_quote_ok:
    print("\n--- Recomendaciones por Co-Cotización ---")
    start_rec_time_quote = time.time()
    recs_quote = get_co_quotation_recommendations(example_product_id, N=5)
    end_rec_time_quote = time.time()
    if not recs_quote.empty:
        print(recs_quote)
        print(f"Tiempo: {end_rec_time_quote - start_rec_time_quote:.4f} seg.")
    else:
        print("No se encontraron recomendaciones.")



--- 12. Ejemplo de Uso de Recomendaciones Colaborativas ---

Obteniendo recomendaciones CF para: 'producto_125'

--- Recomendaciones por Co-Compra ---
         producto  co_purchase_count
726   producto_49                453
634   producto_40                339
255   producto_19                261
225  producto_176                241
98   producto_129                166
Tiempo: 0.0126 seg.

--- Recomendaciones por Co-Cotización ---
Producto 'producto_125' no tiene co-cotizaciones registradas.
No se encontraron recomendaciones.


# 13. Definiendo Función para Obtener Top-N por Método Individual

Se define la función `get_top_n_candidates_per_method` que obtiene las listas separadas de los Top-N productos recomendados según cada método individual (contenido, co-compra, co-cotización). La función recibe como entrada el ID de un producto y el número de recomendaciones a obtener para cada método.


In [ ]:
def get_top_n_candidates_per_method(input_product, N=10):
    """
    Obtiene las listas separadas de los Top-N productos recomendados según
    cada método individual (contenido, co-compra, co-cotización).

    Args:
        input_product (str): El ID del producto de entrada.
        N (int): Número de recomendaciones a obtener para cada método.

    Returns:
        dict: Un diccionario donde las claves son 'content', 'co_purchase', 'co_quotation'
              y los valores son DataFrames con las columnas ['producto', 'score'].
              Los DataFrames estarán vacíos si el método falla o no hay recomendaciones.
    """
    results = {
        'content': pd.DataFrame(columns=['producto', 'score']),
        'co_purchase': pd.DataFrame(columns=['producto', 'score']),
        'co_quotation': pd.DataFrame(columns=['producto', 'score'])
    }

    # --- Validación Inicial ---
    if input_product not in product_to_idx:
        print(f"Error: Producto '{input_product}' no encontrado en el mapeo.")
        return results # Devuelve dicc con DFs vacíos

    idx = product_to_idx[input_product]

    # --- 1. Top-N por Contenido ---
    if similarity_ok and content_similarity_matrix is not None:
        try:
            sim_scores = list(enumerate(content_similarity_matrix[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

            content_recs = []
            for i, score in sim_scores:
                if i == idx: continue
                prod_name = idx_to_product.get(i)
                if prod_name and len(content_recs) < N:
                    content_recs.append({'producto': prod_name, 'score': score})
                elif len(content_recs) >= N:
                    break # Ya tenemos N

            if content_recs:
                results['content'] = pd.DataFrame(content_recs)
        except Exception as e:
            print(f"Error obteniendo Top-N de contenido: {e}")

    # --- 2. Top-N por Co-Compra ---
    if cf_buy_ok and co_occurrence_matrix is not None:
        try:
            co_buys = co_occurrence_matrix[idx, :]
            co_buy_indices = co_buys.indices
            co_buy_values = co_buys.data

            if len(co_buy_indices) > 0:
                cobuy_recs = []
                # Crear lista de tuplas (índice, count) y ordenar
                cobuy_pairs = sorted(zip(co_buy_indices, co_buy_values), key=lambda x: x[1], reverse=True)

                for i, count in cobuy_pairs:
                    if i == idx: continue
                    prod_name = idx_to_product.get(i)
                    if prod_name and len(cobuy_recs) < N:
                        # Guardamos el *conteo* como score aquí
                        cobuy_recs.append({'producto': prod_name, 'score': count})
                    elif len(cobuy_recs) >= N:
                        break

                if cobuy_recs:
                     results['co_purchase'] = pd.DataFrame(cobuy_recs)
        except Exception as e:
            print(f"Error obteniendo Top-N de co-compra: {e}")

    # --- 3. Top-N por Co-Cotización ---
    if cf_quote_ok and co_quotation_matrix is not None:
        try:
            co_quotes = co_quotation_matrix[idx, :]
            co_quote_indices = co_quotes.indices
            co_quote_values = co_quotes.data

            if len(co_quote_indices) > 0:
                coquote_recs = []
                coquote_pairs = sorted(zip(co_quote_indices, co_quote_values), key=lambda x: x[1], reverse=True)

                for i, count in coquote_pairs:
                    if i == idx: continue
                    prod_name = idx_to_product.get(i)
                    if prod_name and len(coquote_recs) < N:
                         # Guardamos el *conteo* como score aquí
                        coquote_recs.append({'producto': prod_name, 'score': count})
                    elif len(coquote_recs) >= N:
                        break

                if coquote_recs:
                    results['co_quotation'] = pd.DataFrame(coquote_recs)
        except Exception as e:
            print(f"Error obteniendo Top-N de co-cotización: {e}")

    return results



# Fórmula para calcular el score final con el método híbrido

La fórmula para calcular el score de un producto `p` usando un enfoque híbrido de recomendaciones es:


$$
\text{score\_hibrido}(p) = w_{\text{content}} \cdot \frac{1}{\text{rank}_{\text{content}}(p) + k} + w_{\text{buy}} \cdot \frac{1}{\text{rank}_{\text{buy}}(p) + k} + w_{\text{quote}} \cdot \frac{1}{\text{rank}_{\text{quote}}(p) + k}
$$
## Donde:

- **rank_content(p)**: Posición (rango) del producto `p` en la lista de recomendaciones por contenido.  
  (1 si es el primero, 2 si es el segundo, etc.). Si `p` no está en la lista, se puede asignar un rango infinito o simplemente un score de 0 para ese componente.

- **rank_buy(p)**: Rango de `p` en la lista de recomendaciones por co-compra.

- **rank_quote(p)**: Rango de `p` en la lista de recomendaciones por co-cotización.

- **w_content, w_buy, w_quote**: Pesos asignados a cada uno de los métodos (contenido, co-compra y co-cotización), que determinan su importancia relativa.

- **k**: Constante pequeña (por ejemplo, 60 como en el paper original de RRF, o valores menores como 1 o 2).  
  Se usa para evitar que los productos en las primeras posiciones tengan un peso desproporcionado y para otorgar algo de valor a los productos que aparecen en posiciones más bajas.  
  Un valor de `k` más alto suaviza las diferencias entre los primeros puestos.


En el modelo híbrido de recomendación, se combinan tres enfoques: basado en contenido, co-compra (colaborativo por transacciones) y co-cotización (colaborativo por cotizaciones). Cada uno aporta una puntuación a los productos candidatos, y estas se combinan ponderadamente mediante tres pesos: w_content, w_buy y w_quote, que reflejan la importancia relativa de cada fuente de información. Los pesos permiten controlar cuánto influye cada tipo de recomendación en el resultado final. Además, para mejorar la robustez del sistema ante casos de baja información colaborativa, se implementa una lógica adaptativa: si el producto de entrada tiene un score bajo de co-compra (por ejemplo, menos de 20), se considera que hay poca evidencia útil desde las transacciones, por lo que se incrementa el peso del modelo basado en contenido (por ejemplo, w_content = 0.5, w_buy = 0.3, w_quote = 0.2). Esto permite que el sistema se apoye más en las características del producto cuando el historial de interacción es escaso, mejorando así la calidad y relevancia de las recomendaciones.

In [ ]:
def get_recommendations_hybrid_rerank(input_product, N=10,
                                      content_weight=0.3,
                                      cf_buy_weight=0.5,
                                      cf_quote_weight=0.2,
                                      k=2, # Constante para suavizar RRF-like score
                                      fetch_top_M=50): # Cuántos candidatos obtener de cada método
    """
    Genera recomendaciones híbridas usando re-ranking basado en la posición
    en las listas de cada método individual.

    Args:
        input_product (str): El ID del producto de entrada.
        N (int): Número de recomendaciones finales deseadas.
        content_weight (float): Peso para la importancia del ranking de contenido.
        cf_buy_weight (float): Peso para la importancia del ranking de co-compra.
        cf_quote_weight (float): Peso para la importancia del ranking de co-cotización.
        k (int): Constante de suavizado para el cálculo del score (mayor k, más suave).
        fetch_top_M (int): Cuántos candidatos obtener inicialmente de cada método individual.

    Returns:
        pandas.DataFrame: Top N recomendaciones con 'producto' y 'hybrid_score'.
                          DF vacío si hay error o el producto no existe.
    """
    hybrid_scores = defaultdict(float)

    # --- 1. Obtener listas Top-M de cada método ---
    # Pedimos más candidatos (Top-M) para tener una base más amplia para el re-ranking
    separate_candidates = get_top_n_candidates_per_method(input_product, N=fetch_top_M)

    #Si el producto objetivo tiene un score bajo de co-compra 
    # (menos de 20 compras conjuntas con otros productos), se 
    # considera que el modelo colaborativo por co-compra no tiene 
    # suficiente información para ser confiable. Por tanto, se incrementa 
    # el peso del modelo basado en contenido, porque este no depende de 
    # interacciones pasadas, sino de las características del producto en sí.
    candida_productos_co_purchase = separate_candidates['co_purchase']['score'][0]
    if candida_productos_co_purchase < 10:
        content_weight=0.5
        cf_buy_weight=0.3
        cf_quote_weight=0.2
    # --- 2. Calcular Score Híbrido basado en Rango Inverso Ponderado ---
    all_candidates = set() # Conjunto de todos los productos únicos recomendados

    # Recolectar todos los candidatos y calcular scores
    for method, weight in [('content', content_weight),
                           ('co_purchase', cf_buy_weight),
                           ('co_quotation', cf_quote_weight)]:

        if weight <= 0: continue # Saltar si el peso es cero o negativo

        recs_df = separate_candidates.get(method)
        if recs_df is not None and not recs_df.empty:
            # Añadir candidatos al conjunto general
            all_candidates.update(recs_df['producto'].tolist())
            # Calcular contribución al score híbrido para cada producto en esta lista
            for rank, product in enumerate(recs_df['producto'], 1): # Rank empieza en 1
                 # Score = peso * (1 / (rango + k))
                 score_contribution = weight * (1.0 / (rank + k))
                 hybrid_scores[product] += score_contribution

    # --- 3. Generar Ranking Final ---
    if not hybrid_scores:
        print(f"No se encontraron candidatos para hibridar para '{input_product}'.")
        return pd.DataFrame()

    # Convertir scores híbridos a DataFrame
    final_recs_df = pd.DataFrame(hybrid_scores.items(), columns=['producto', 'hybrid_score'])

    # Asegurarse de no incluir el producto de entrada
    final_recs_df = final_recs_df[final_recs_df['producto'] != input_product]

    # Ordenar por el nuevo score híbrido y tomar Top N
    final_recs_df = final_recs_df.sort_values('hybrid_score', ascending=False).head(N)

    return final_recs_df



# 15. Ejemplo de Uso de la Recomendación Híbrida por Re-Ranking

En este paso, se realiza un ejemplo de cómo obtener recomendaciones híbridas utilizando el método de re-ranking. Para ello, se ajustan los pesos y parámetros que determinan la influencia de cada método de recomendación (contenido, co-compra y co-cotización). La función `get_recommendations_hybrid_rerank` es utilizada para obtener las recomendaciones finales basadas en estos pesos y se imprime el tiempo total del proceso.

In [ ]:

# Usar el mismo ID de producto de ejemplo
example_product_id = "producto_2"

if example_product_id:
    start_time_rerank = time.time()
    
    print(f"\nObteniendo recomendaciones HÍBRIDAS (Re-Rank) para: '{example_product_id}'")
    start_rec_time_rerank = time.time()

    # --- AJUSTA LOS PESOS Y PARÁMETROS AQUÍ ---
    recommendations = get_recommendations_hybrid_rerank(
        example_product_id,
        N=10,            # Pedir 10 recomendaciones finales
        fetch_top_M=50,  # Obtener 50 candidatos iniciales de cada método
        k=1,            # Constante de suavizado RRF (valor común)
        content_weight=0.3,
        cf_buy_weight=0.5,
        cf_quote_weight=0.2 # Si co-cotización no dio resultados, este peso será inefectivo
    )

    end_rec_time_rerank = time.time()

    if not recommendations.empty:
        print("\n--- Recomendaciones Híbridas Finales (Re-Rank) ---")
        print(recommendations)
        print(f"\nTiempo total de recomendación híbrida (Re-Rank): {end_rec_time_rerank - start_time_rerank:.4f} segundos.")
    else:
        print("No se generaron recomendaciones híbridas para el producto de ejemplo.")

else:
    print("\nNo se puede ejecutar el ejemplo híbrido: 'example_product_id' no está definido o no es válido.")

print("\n--- Proceso de Modelo Híbrido (Re-Ranking) Completado ---")


--- 16. Ejemplo: Recomendación Híbrida por Re-Ranking ---

Obteniendo recomendaciones HÍBRIDAS (Re-Rank) para: 'producto_2'

--- Recomendaciones Híbridas Finales (Re-Rank) ---
         producto  hybrid_score
50  producto_3126      0.350000
1   producto_1120      0.234524
0   producto_1232      0.213462
51   producto_413      0.166667
43   producto_632      0.118095
19  producto_1315      0.116667
52   producto_119      0.100000
53   producto_110      0.083333
9     producto_17      0.082828
59   producto_208      0.077778

Tiempo total de recomendación híbrida (Re-Rank): 0.0240 segundos.

--- Proceso de Modelo Híbrido (Re-Ranking) Completado ---


# Extra. Ejemplo de Uso de la Recomendación de los metodos totales

Aca se muestra las recomendaciones para un producto y que suelta cada modelo por separado y se puede comparar asi con el resultado por el modelo hibrido para comparar y ver su funcionamiento

In [24]:
example_product_id = "producto_2"

# --- 8. Ejemplo de Uso ---
if similarity_ok and all_unique_products:
    

    if example_product_id:
        print(f"Obteniendo recomendaciones de contenido para: '{example_product_id}'")
        start_rec_time = time.time()
        recs = get_content_recommendations(example_product_id, N=5)
        end_rec_time = time.time()

        if not recs.empty:
            print(recs)
        else:
            print("No se generaron recomendaciones para el producto de ejemplo.")


else:
    print("No se puede ejecutar el ejemplo: la matriz de similitud no está lista o no hay productos.")


# Ejemplo Co-Compra
if cf_buy_ok:
    print("\n--- Recomendaciones por Co-Compra ---")
    start_rec_time_buy = time.time()
    recs_buy = get_co_purchase_recommendations(example_product_id, N=5)
    end_rec_time_buy = time.time()
    if not recs_buy.empty:
        print(recs_buy)
    else:
        print("No se encontraron recomendaciones.")

# Ejemplo Co-Cotización
if cf_quote_ok:
    print("\n--- Recomendaciones por Co-Cotización ---")
    start_rec_time_quote = time.time()
    recs_quote = get_co_quotation_recommendations(example_product_id, N=5)
    end_rec_time_quote = time.time()
    if not recs_quote.empty:
        print(recs_quote)
    else:
        print("No se encontraron recomendaciones.")


Obteniendo recomendaciones de contenido para: 'producto_2'
        producto  similarity_score
0  producto_1232          0.999869
1  producto_1120          0.999325
2  producto_1110          0.999110
3  producto_3115          0.998920
4   producto_837          0.998911

--- Recomendaciones por Co-Compra ---
          producto  co_purchase_count
266   producto_413                 17
208  producto_3126                 17
20   producto_1120                 16
33    producto_119                 16
330   producto_632                 15

--- Recomendaciones por Co-Cotización ---
          producto  co_quotation_count
68   producto_3126                  13
104   producto_632                   6
39    producto_208                   6
105   producto_642                   6
121   producto_865                   6
